In [101]:
import numpy as np
import collections
import sys
filename = input("enter the filename with the relative path")
data=[i.rstrip().split() for i in open(filename, 'r').readlines()]
data = np.array(data)

enter the filename with the relative pathproject3_dataset1.txt


In [102]:
def isString(data):
    try:
        float(data)
        return False
    except:
        pass
        return True
def preprocessTrainingData(trainingData):
    trainingLen  = trainingData.shape[0]
    trainingbreath  = trainingData.shape[1]
    RealtrainingData = np.empty(shape=(trainingLen,0), dtype='float64')
    NominaltrainingData = np.empty(shape=(trainingLen,0), dtype='<U7')
    for index in range(trainingbreath-1):
        if(isString(trainingData[0,index])):
            NominaltrainingData=np.append(NominaltrainingData,trainingData[:,index].reshape(trainingLen,1),axis=1)
        else:
            RealtrainingData=np.append(RealtrainingData,trainingData[:,index].reshape(trainingLen,1).astype(float),axis=1)
    return RealtrainingData, NominaltrainingData
def preprocessValidationData(validationData):
    validationLen = validationData.shape[0]
    validationbreath = validationData.shape[1]
    RealvalidationData = np.empty(shape=(validationLen,0), dtype='float64')
    Nominalvalidation = np.empty(shape=(validationLen,0), dtype='<U7')
    for index in range(validationbreath):
        if(isString(validationData[0,index])):
            Nominalvalidation=np.append(Nominalvalidation,validationData[:,index].reshape(validationLen,1),axis=1)
        else:
            RealvalidationData=np.append(RealvalidationData,validationData[:,index].reshape(validationLen,1).astype(float),axis=1)
    return RealvalidationData, Nominalvalidation
def GaussianFormulaReal(validationDataContinuous, means, std, classes):
    valPostContinuous=np.empty(shape=(validationDataContinuous.shape[0],len(classes)))
    for i in range(len(validationDataContinuous)):
            ec=np.exp((-1)*(validationDataContinuous[i] - means).dot((validationDataContinuous[i] - means).T)/
                      (2*(std.dot(std.T))))
            post=(1/(((2*np.pi)**(0.5))*std)).T.dot(ec)
            valPostContinuous[i]=post.prod(0)       
    return valPostContinuous
def GaussianFormulaNominal(validationDataNominal, means, std, classes, NominaltrainingData,trainingLabels ):
    valPostNominal=np.empty(shape=(validationDataNominal.shape[0],len(classes),validationDataNominal.shape[1]))
    
    for i in range(validationDataNominal.shape[0]):
        for j in range(len(classes)):
            for k in range(validationDataNominal.shape[1]):
                count  = 0
                for label in NominaltrainingData[np.array(trainingLabels)==classes[j]][:,k]:
                    if(label == validationDataNominal[i,k] ):
                        count = count+1
                p =(count/(NominaltrainingData.shape[0]))*1.0
                valPostNominal[i,j,k]=p
    return valPostNominal
    
    

In [103]:
def NaiveBayes(tData, vData, tLabels, vLabels):

    RealtrainingData, NominaltrainingData = preprocessTrainingData(tData)
    
    RealvalidationData, Nominalvalidation = preprocessValidationData(vData)
    
    means=np.zeros(shape=(len(np.unique(tLabels)),RealtrainingData.shape[1]))
    std=np.zeros(shape=(len(np.unique(tLabels)),RealtrainingData.shape[1]))

    classes=np.unique(np.array(tLabels))

    for i in range(len(classes)):            
        means[i]=RealtrainingData[np.array(tLabels)==classes[i]].mean(0)
        std[i]=RealtrainingData[np.array(tLabels)==classes[i]].std(0)
        
    Realdata = GaussianFormulaReal(RealvalidationData, means, std, classes)
    Nominaldata =  GaussianFormulaNominal(Nominalvalidation, means, std, classes, NominaltrainingData,tLabels)
    
#     for i in range(len(Realdata)):
#         print("p(H0|X)"+str((Nominaldata.prod(2)*Realdata)[i,0])+"\t"+"p(H1|X)"+str((Nominaldata.prod(2)*Realdata)[i,1]))
#     print((Nominaldata.prod(2),Realdata))
    predictedLabels=(Nominaldata.prod(2)*Realdata).argmax(1)
    validationLabels=np.array(vLabels).astype(float)
    
    return predictedLabels, validationLabels

In [104]:
randomdata = np.random.permutation(len(data))
accuracies = np.zeros(shape=(K,))
precision = np.zeros(shape=(K,))
recall = np.zeros(shape=(K,))
f1 = np.zeros(shape=(K,))

In [105]:
def metrics(predictedLabels,trueLabels):
    a=0
    b=0
    c=0
    d=0
    for i, j in zip(trueLabels,predictedLabels):
            if ((j == 1) & (i == j)):
                a += 1
            elif ((j == 0) & (i != j)):
                b += 1
            elif ((j == 1) & (i != j)):
                c += 1
            elif ((j == 0) & (i == j)):
                d += 1
    sum = a+b+c+d
    if sum == 0:
        accuracy = "Not possible to compute"
    else:
        accuracy = (a+d)/sum
    if a+c == 0:
        precision = "Not possible to compute"
    else:
        precision = a/(a+c)
    if a+b == 0:
        recall = "Not possible to compute"
    else:
        recall = a/(a+b)
    if a+b+c == 0:
        f1 = "Not possible to compute"
    else:
        f1 = 2*a/(2*a+b+c)
    return accuracy,precision,recall,f1

In [106]:

randomdata = np.random.permutation(len(data))
length = len(randomdata)
Allaccuracies = []
Allprecision = []
Allrecall =[]
Allf1score = []
length = len(randomdata)
for i in range(10):
    tdata = []
    tdataLabels = []
    vdata = []
    vdataLabels = []
    start = i *length//10
    end = (i+1)*length//10
    for j in range(length):
        if(j in range(start,end)):
            vdata.append(data[randomdata[j]][:-1])
            vdataLabels.append(data[randomdata[j]][-1])
        else:
            tdata.append(data[randomdata[j]])
            tdataLabels.append(data[randomdata[j]][-1])
    tdata = np.array(tdata)
    vdata = np.array(vdata)
    actualLabels, predictedLabels  = NaiveBayes(tdata, vdata, tdataLabels, vdataLabels)
    actualLabels = np.array(actualLabels).astype(int)
    predictedLabels = np.array(predictedLabels).astype(int)
    accuracy,precision,recall,f1 = metrics(predictedLabels,actualLabels)
    if not (isString(accuracy)):
        Allaccuracies.append(accuracy)
    if not (isString(precision)):
        Allprecision.append(precision)
    if not (isString(recall)):
        Allrecall.append(recall)
    if not (isString(f1)):
        Allf1score.append(f1)
    print("\nIteration " + str(i))
    print("Training data Length: " + str(len(tdata)))
    print("Validation data Length: " + str(len(vdata)))
    print("accuracy " + str(accuracy))
    print("precision " + str(precision))
    print("recall " + str(recall))
    print("f1 " + str(f1))
print("\nAverageaccuracy " + str(np.array(Allaccuracies).mean()))
print("Averageprecision " + str(np.array(Allprecision).mean()))
print("Averagerecall " + str(np.array(Allrecall).mean()))
print("Averagef1 " + str(np.array(Allf1score).mean()))


Iteration 0
Training data Length: 513
Validation data Length: 56
accuracy 0.8392857142857143
precision 1.0
recall 0.7
f1 0.8235294117647058

Iteration 1
Training data Length: 512
Validation data Length: 57
accuracy 0.8947368421052632
precision 0.9545454545454546
recall 0.8076923076923077
f1 0.875

Iteration 2
Training data Length: 512
Validation data Length: 57
accuracy 0.7543859649122807
precision 1.0
recall 0.5483870967741935
f1 0.7083333333333334

Iteration 3
Training data Length: 512
Validation data Length: 57
accuracy 0.8070175438596491
precision 1.0
recall 0.65625
f1 0.7924528301886793

Iteration 4
Training data Length: 512
Validation data Length: 57
accuracy 0.9298245614035088
precision 1.0
recall 0.8333333333333334
f1 0.9090909090909091

Iteration 5
Training data Length: 512
Validation data Length: 57
accuracy 0.7719298245614035
precision 0.9629629629629629
recall 0.6842105263157895
f1 0.8

Iteration 6
Training data Length: 512
Validation data Length: 57
accuracy 0.84210526315